In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress


In [2]:
# Reading CSV files into dataframes
daily_activity_df = pd.read_csv("dailyActivity_merged (1).csv")
fitbit_df = pd.read_csv("FitBit data.csv")

#check DataFrames 
daily_activity_df.head()

  

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [3]:
fitbit_df.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,3/25/2016,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,3/26/2016,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154
2,1503960366,3/27/2016,12736,8.53,8.53,0.0,4.66,0.16,3.71,0.0,56,5,268,605,1944
3,1503960366,3/28/2016,13231,8.93,8.93,0.0,3.19,0.79,4.95,0.0,39,20,224,1080,1932
4,1503960366,3/29/2016,12041,7.85,7.85,0.0,2.16,1.09,4.61,0.0,28,28,243,763,1886


In [4]:
#Print the column names
print(daily_activity_df.columns)
print(fitbit_df .columns)   

Index(['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance',
       'LoggedActivitiesDistance', 'VeryActiveDistance',
       'ModeratelyActiveDistance', 'LightActiveDistance',
       'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories'],
      dtype='object')
Index(['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance',
       'LoggedActivitiesDistance', 'VeryActiveDistance',
       'ModeratelyActiveDistance', 'LightActiveDistance',
       'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories'],
      dtype='object')


In [5]:
#merging the DataFrames
merged_df = pd.concat([daily_activity_df, fitbit_df], ignore_index=True)
merged_df.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [6]:
# Print the number of rows in the merged dataset
print("merged_df:", len(merged_df))

merged_df: 1397


In [11]:
# find the duplicate rows by  ID, activity date and total distance
duplicate_df = merged_df[merged_df.duplicated(subset=["Id", "ActivityDate", "TotalSteps"])]

# Remove duplicates from the original data based on specific columns
cleaned_data = merged_df.drop_duplicates(subset=['Id', 'ActivityDate', 'TotalSteps'])

#duplicate_df.to_csv("duplicates.csv", index=False)
#cleaned_data.to_csv("cleaned_data.csv", index=False)

In [12]:
# Define recommended guidelines
# rec dail activity in mins
rec_mins_pw = 90

# Rec daily steps
rec_daily_steps = 10000



In [13]:
# Ensure the activity_date column is in datetime format
cleaned_data["ActivityDate"] = pd.to_datetime(cleaned_data["ActivityDate"])

# Calculate the total number of activity dates for each ID
activity_counts = cleaned_data.groupby("Id")["ActivityDate"].nunique().reset_index()
activity_counts.columns = ["Id", "TotalActivityDates"]

# Calculate the total of all other columns for each ID
# Exclude the 'activity_date' column for the sum operation
columns_to_sum = cleaned_data.columns.difference(["ActivityDate"])

column_sums = cleaned_data[columns_to_sum].groupby("Id").sum().reset_index()

# Merge the two results
result = pd.merge(activity_counts, column_sums, on="Id")

# Rename columns to include the word 'Total' in each column header
result.columns = ['Sum' + col if col != 'Id' else col for col in result.columns]

# Display the final result
print(result)

            Id  SumTotalActivityDates  SumCalories  SumFairlyActiveMinutes  \
0   1503960366                     49        90437                     895   
1   1624580081                     49        71689                     191   
2   1644430081                     40       113503                    1076   
3   1844505072                     42        68169                      49   
4   1927972279                     42        94405                      44   
5   2022484408                     42       107513                     870   
6   2026352035                     42        64026                       8   
7   2320127002                     42        71834                      93   
8   2347167796                     32        67102                     717   
9   2873212765                     42        79775                     262   
10  2891001357                      8        18187                     660   
11  3372868164                     30        57265              

In [14]:
# Add new column for total active minutes
result["TotalActiveMinutes"] = result["SumFairlyActiveMinutes"] + result["SumLightlyActiveMinutes"] + result["SumVeryActiveMinutes"]

# Add new column for total active distance
result["TotalActiveDistance"] = result["SumLightActiveDistance"] + result["SumModeratelyActiveDistance"] + result["SumVeryActiveDistance"]

# Display the final result with new columns
print(result)


            Id  SumTotalActivityDates  SumCalories  SumFairlyActiveMinutes  \
0   1503960366                     49        90437                     895   
1   1624580081                     49        71689                     191   
2   1644430081                     40       113503                    1076   
3   1844505072                     42        68169                      49   
4   1927972279                     42        94405                      44   
5   2022484408                     42       107513                     870   
6   2026352035                     42        64026                       8   
7   2320127002                     42        71834                      93   
8   2347167796                     32        67102                     717   
9   2873212765                     42        79775                     262   
10  2891001357                      8        18187                     660   
11  3372868164                     30        57265              

In [15]:
# Calculate the average of each column
averages = result.mean()
# Exclude the Id column
averages_excluding_id = averages.drop("Id")
# Format the values to 4 decimal places
averages_formatted = averages_excluding_id.round(4)
print(averages_formatted)

SumTotalActivityDates              39.2286
SumCalories                     90456.3714
SumFairlyActiveMinutes            534.9714
SumLightActiveDistance            127.4626
SumLightlyActiveMinutes          7399.0286
SumLoggedActivitiesDistance         5.2480
SumModeratelyActiveDistance        21.4923
SumSedentaryActiveDistance          0.0680
SumSedentaryMinutes             39616.6286
SumTotalDistance                  208.3300
SumTotalSteps                  290611.8571
SumTrackerDistance                207.2437
SumVeryActiveDistance              55.7769
SumVeryActiveMinutes              785.4857
TotalActiveMinutes               8719.4857
TotalActiveDistance               204.7317
dtype: float64


Here I want to see if each ID met the daily recommended activity time and total step counts

In [18]:
# Calculate average daily active minutes and daily steps
result['AverageDailyActiveMinutes'] = result['TotalActiveMinutes'] / result['SumTotalActivityDates']
result['AverageDailySteps'] = result['SumTotalSteps'] / result['SumTotalActivityDates']

# Determine if they met the recommended daily active minutes and step count
result['Met daily rec activity mins'] = result['AverageDailyActiveMinutes'] >= rec_mins_pw
result['Met daily rec step count'] = result['AverageDailySteps'] >= rec_daily_steps

# Convert boolean values to 'TRUE'/'FALSE'
result['Met daily rec activity mins'] = result['Met daily rec activity mins'].apply(lambda x: 'TRUE' if x else 'FALSE')
result['Met daily rec step count'] = result['Met daily rec step count'].apply(lambda x: 'TRUE' if x else 'FALSE')

# Display the final result
print(result)

# convert to csv
#result.to_csv("result.csv", index=False)

            Id  SumTotalActivityDates  SumCalories  SumFairlyActiveMinutes  \
0   1503960366                     49        90437                     895   
1   1624580081                     49        71689                     191   
2   1644430081                     40       113503                    1076   
3   1844505072                     42        68169                      49   
4   1927972279                     42        94405                      44   
5   2022484408                     42       107513                     870   
6   2026352035                     42        64026                       8   
7   2320127002                     42        71834                      93   
8   2347167796                     32        67102                     717   
9   2873212765                     42        79775                     262   
10  2891001357                      8        18187                     660   
11  3372868164                     30        57265              